In [10]:
from utils_directory_tree_generator import get_directory_tree

tree_generator = get_directory_tree(
    start_path='/kaggle/input',
    max_depth=5,
    include_files=True,
    sort_by='type',
    reverse=False,
    max_items=5
)
for line in tree_generator:
    print(line)

input
    google-research-identify-contrails-reduce-global-warming
        validation
            3687499407028137410
                band_10.npy
                band_14.npy
                band_15.npy
                band_16.npy
                band_08.npy
            6558861185867890815
                band_10.npy
                band_14.npy
                band_15.npy
                band_16.npy
                band_08.npy
            7355354609194882312
                band_10.npy
                band_14.npy
                band_15.npy
                band_16.npy
                band_08.npy
            7547747455642200110
                band_10.npy
                band_14.npy
                band_15.npy
                band_16.npy
                band_08.npy
            5456834089979970017
                band_10.npy
                band_14.npy
                band_15.npy
                band_16.npy
                band_08.npy
        test
            1002653297254493116
         

In [11]:
! pip install -q segmentation-models-pytorch --upgrade

In [12]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda, Resize, RandomAffine, ColorJitter
from torch.nn.functional import pad
from PIL import Image
from utils_contrails_loss_functions import DiceLoss, FocalLoss
import segmentation_models_pytorch as smp
import pytorch_lightning as pl

# Dataset paths
train_data_path = "/kaggle/input/google-research-identify-contrails-reduce-global-warming/train"
val_data_path = "/kaggle/input/google-research-identify-contrails-reduce-global-warming/validation"
test_data_path = "/kaggle/input/google-research-identify-contrails-reduce-global-warming/test"


# Data Loader
class ContrailDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.record_dirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.record_dirs.sort(key=lambda x: os.path.getsize(x)) # Sort dataset based on sequence length

        # Updated transformations
        self.transform = Compose([
            Lambda(lambda x: Image.fromarray((x.transpose(1, 2, 0) * 255).astype(np.uint8))),
            ColorJitter(brightness=0.1, contrast=0.1),
            Lambda(lambda x: np.array(x)),
            Lambda(lambda x: x.transpose(2, 0, 1)),
            Lambda(lambda x: torch.from_numpy(x).float()),
            RandomAffine(degrees=0, scale=(0.8, 1.2), shear=20),
            Resize((320, 320)),
            Lambda(lambda x: pad(x, (0, 0, 0, (32 - x.shape[1] % 32) % 32)))
            
        ])
        self.target_transform = Compose([
            Lambda(lambda x: torch.from_numpy(x).float())
        ])

    def __len__(self):
        return len(self.record_dirs)

    def __getitem__(self, idx):
        record_dir = self.record_dirs[idx]
        
        # Updated to use bands from 13 to 16 (4 bands)
        bands = [np.load(os.path.join(record_dir, f"band_{band}.npy"))[..., 0] for band in [13, 14, 15, 16]]
        bands_array = np.stack(bands)
        sample = bands_array

        # Load aggregated ground truth mask
        target_path = os.path.join(record_dir, 'human_pixel_masks.npy')
        target = np.load(target_path) if os.path.exists(target_path) else None
        
        # Apply the target transformations to the target
        if target is not None:
            # Ensure the target has only one channel and appropriate shape.
            target = np.mean(target, axis=(0, 1))
            
            # Ensure the data type is uint8 and values range between 0 and 255
            target = (target * 255).astype(np.uint8)
            
            # Convert numpy array to PIL Image, then resize, and convert back to numpy array
            target_pil = Image.fromarray(target)
            
            # Resize the target to 256x256 to match the model's output
            target_resized = np.array(Resize((256, 256))(target_pil))
        
        # Convert to PyTorch tensors
        sample = torch.tensor(sample, dtype=torch.float32)
        if target is not None:
            target = torch.tensor(target_resized, dtype=torch.float32).unsqueeze(0)  # Add channel dimension

        return sample, target
        
class CombinedLoss(nn.Module):
    def __init__(self):
        super(CombinedLoss, self).__init__()
        self.dice_loss = DiceLoss()
        self.focal_loss = FocalLoss()
    
    def forward(self, inputs, targets):
        dice = self.dice_loss(inputs, targets)
        focal = self.focal_loss(inputs, targets)
        return dice + focal

class ContrailModel(pl.LightningModule):
    def __init__(self, in_channels=4):
        super(ContrailModel, self).__init__()
        self.model = smp.Unet(
            encoder_name="resnet34",
            encoder_weights="imagenet",
            in_channels=in_channels,
            classes=1,
            activation=None
        )
        self.loss = CombinedLoss()  # Instantiate the combined loss
        self.sigmoid = nn.Sigmoid()  # Add sigmoid activation layer

    def forward(self, x):
        x = self.model(x)
        return self.sigmoid(x)  # Apply sigmoid activation before returning


    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        print(y_hat.shape)
        print(y.shape)
        loss_val = self.loss(y_hat, y)  #  compute combined loss
        self.log('train_loss', loss_val)
        return loss_val


    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.001)

# Instantiate datasets and model
dataset_train = ContrailDataset(root_dir=train_data_path)
dataset_val = ContrailDataset(root_dir=val_data_path)

model = ContrailModel(in_channels=4)

# Training Function
def train_model(train_dataset, val_dataset, model, epochs=10, batch_size=8, lr=0.001):
    # Use the collate function in data loader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = BCEWithLogitsLoss()
    
    for epoch in range(epochs):
        model.train()
        for images, masks in train_loader: 
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks) 
            loss.backward()
            optimizer.step()
        
        model.eval()
        val_loss = sum(criterion(model(images), masks).item() for images, masks in val_loader) / len(val_loader)
        print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {val_loss:.4f}")

# Instantiate datasets
dataset_train = ContrailDataset(root_dir=train_data_path)
dataset_val = ContrailDataset(root_dir=val_data_path)

# Instantiate model
model = ContrailModel(in_channels=4)


# Use PyTorch Lightning's Trainer to train the model
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, DataLoader(dataset_train, batch_size=8, shuffle=True), DataLoader(dataset_val, batch_size=8))

Training: 0it [00:00, ?it/s]

torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])
torch.Size([8, 1, 256, 256])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:159                                                                                  │
│                                                                                                  │
│   156                                                                                            │
│   157 # Use PyTorch Lightning's Trainer to train the model                                       │
│   158 trainer = pl.Trainer(max_epochs=10)                                                        │
│ ❱ 159 trainer.fit(model, DataLoader(dataset_train, batch_size=8, shuffle=True), DataLoader(dat   │
│   160                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:531 in fit          │
│                                                                                                  │
│    528 │   │   """                                                                               │
│    529 │   │   model = _maybe_unwrap_optimized(model)                                            │
│    530 │   │   self.strategy._lightning_module = model                                           │
│ ❱  531 │   │   call._call_and_handle_interrupt(                                                  │
│    532 │   │   │   self, self._fit_impl, model, train_dataloaders, val_dataloaders, datamodule,  │
│    533 │   │   )                                                                                 │
│    534                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:42 in                  │
│ _call_and_handle_interrupt                                                                       │
│                                                                                                  │
│    39 │   try:                                                                                   │
│    40 │   │   if trainer.strategy.launcher is not None:                                          │
│    41 │   │   │   return trainer.strategy.launcher.launch(trainer_fn, *args, trainer=trainer,    │
│ ❱  42 │   │   return trainer_fn(*args, **kwargs)                                                 │
│    43 │                                                                                          │
│    44 │   except _TunerExitException:                                                            │
│    45 │   │   _call_teardown_hook(trainer)                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:570 in _fit_impl    │
│                                                                                                  │
│    567 │   │   │   model_provided=True,                                                          │
│    568 │   │   │   model_connected=self.lightning_module is not None,                            │
│    569 │   │   )                                                                                 │
│ ❱  570 │   │   self._run(model, ckpt_path=ckpt_path)                                             │
│    571 │   │                                                                                     │
│    572 │   │   assert self.state.stopped                                                         │
│    573 │   │   self.training = False                                                             │
│                                                            

In [ ]:
import segmentation_models_pytorch
print(segmentation_models_pytorch.__version__)
